In [1]:

!pip install selenium

!apt-get update

!apt install chromium-chromedriver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubu

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

# Chrome 드라이버 옵션 설정
options = webdriver.ChromeOptions()

# 코랩에서는 headless 모드(화면에 브라우저가 표시되지 않는 모드)를 사용하면 안정적으로 실행됩니다.
options.add_argument('--headless') # 브라우저를 표시하지 않는 headless 모드로 실행합니다.
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def get_news_by_theme(theme_url, date):
    # 뉴스 테마와 본문을 저장할 리스트 초기화
    themes = []
    news_contents = []

    # 웹 드라이버 시작
    driver = webdriver.Chrome(options=options)

    # 대상 페이지로 이동
    driver.get(theme_url.format(date))
    time.sleep(3)

    # 10개의 뉴스 링크 가져오기
    news_links = []
    for i in range(1, 11):
        news_xpath = f'//*[@id="contents"]/div/div/div[2]/section/ul/li[{i}]/article/div/h4/a'
        news_element = driver.find_element(By.XPATH, news_xpath)
        news_link = news_element.get_attribute("href")
        news_links.append(news_link)

    # 10개의 뉴스 테마와 본문 내용 가져오기
    for news_link in news_links:
        driver.get(news_link)
        time.sleep(2)  # 페이지 로드를 기다리기 위해 충분한 시간을 줍니다.

        try:
            # 뉴스 테마 가져오기
            news_theme_element = driver.find_element(By.XPATH, '//*[@id="contents"]/header/div/section/nav/ol/li/a')
            news_theme = news_theme_element.text
            themes.append(news_theme)

            # 뉴스 본문 내용 가져오기
            news_content_element = driver.find_element(By.XPATH, "//*[@id=\"contents\"]/div[2]/div/div[1]/section[1]")
            news_content = news_content_element.text
            news_contents.append(news_content)

        except:
            print("테마나 본문 내용을 가져올 수 없습니다.")

    # 웹 드라이버 종료
    driver.quit()

    # 데이터프레임으로 변환하여 반환
    df = pd.DataFrame({"Theme": themes, "News Content": news_contents})
    return df

# 대상 페이지들의 URL
urls = {
    "Politics": "https://www.donga.com/news/Politics/List?ymd={}&m=",
    "Economy": "https://www.donga.com/news/Economy/List?ymd={}&m=",
    "International": "https://www.donga.com/news/Inter/List?ymd={}&m=",
    "Society": "https://www.donga.com/news/Society/List?ymd={}&m=",
}

# 날짜 설정
start_date = datetime(2023, 6, 1)
end_date = start_date + timedelta(days=14)  # 15일치

# 날짜별로 가져온 데이터프레임을 저장할 리스트 초기화
all_dfs = []

# 각 날짜에 대해 뉴스 가져오기
for single_date in pd.date_range(start=start_date, end=end_date):
    date_str = single_date.strftime("%Y%m%d")
    print("날짜:", date_str)

    # 각 테마별로 뉴스 가져오기
    dfs = []
    for theme, url in urls.items():
        print(f"{theme} 뉴스:")
        df = get_news_by_theme(url, date_str)
        dfs.append(df)
        print(df)
        print("="*50)

    # 각 날짜별 데이터프레임을 합쳐서 리스트에 저장
    all_dfs.append(pd.concat(dfs, ignore_index=True))

# 모든 날짜별 데이터프레임을 하나로 합치기
all_news_df = pd.concat(all_dfs, ignore_index=True)

# 결과 출력
print("전체 뉴스:")
print(all_news_df)


날짜: 20230601
Politics 뉴스:
  Theme                                       News Content
0    정치  장거리지대공미사일(L-SAM) 요격미사일이 발사되는 모습. ADD 제공\n북한이 군...
1    정치  불법 대선 자금 수수 혐의를 받고 있는 김용 전 민주연구원 부원장이 1일 오전 서울...
2    정치  "유력 이용호가 '최고위원 역할 없다'고"\n"민주당 최고위원 보면 좀 비대칭 전력...
3    정치  윤석열 대통령이 1일 서울 강서구 마곡동 ‘서울창업허브M+‘에서 열린 제5차 수출전...
4    정치  가상자산(암호화폐) 거래 논란으로 더불어민주당을 탈당한 김남국 무소속 의원이 31일...
5    사회  경기 파주에서 아프리카돼지열병이 국내 처음으로 발생한 17일 오전 경기도 수원시 팔...
6    정치  김건희 여사가 1일 서울 중구 사랑의열매 회관에서 열린 국가유공자 지원을 위한 기부...
7    정치  카이스트 "국제협력 예산 부족" 호소하자\n尹, 과기부 장관에 "예산 크게 늘려야"...
8    정치  "검찰에서 들여다보겠다면 성실히 임할 것"\n신현영 더불어민주당 의원이 1일 경찰이...
9    정치  노태악 중앙선거관리위원회 위원장이 31일 오후 경기도 과천 선관위에서 고위직 간부 ...
Economy 뉴스:
테마나 본문 내용을 가져올 수 없습니다.
테마나 본문 내용을 가져올 수 없습니다.
테마나 본문 내용을 가져올 수 없습니다.
  Theme                                       News Content
0    사회  경기남부지방경찰청 전경. ⓒ News1\n수도권에 오피스텔 등 100여채를 소유한 ...
1    경제  1998년~2016년 출고 모델 대상\n36가지 항목 무상점검·주요 부품 할인\n르...
2    국제  꾸준한 금리인상 불구 아직도 인플레 전망 만족할 수준 못돼\n크리스틴 라가르드 유럽...
3    경제 

In [4]:
# 모든 뉴스를 모은 데이터프레임을 CSV 파일로 저장
all_news_df.to_csv('all_news.csv', index=False)

In [7]:
# 데이터프레임 테마별 개수 확인
all_news_df['Theme'].value_counts()

사회       168
정치       156
국제       139
경제       100
문화         9
스포츠        8
IT/의학      5
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [8]:
copy_df = all_news_df
copy_df

,Theme,News Content
0,정치,장거리지대공미사일(L-SAM) 요격미사일이 발사되는 모습. ADD 제공\n북한이 군...
1,정치,불법 대선 자금 수수 혐의를 받고 있는 김용 전 민주연구원 부원장이 1일 오전 서울...
2,정치,"""유력 이용호가 '최고위원 역할 없다'고""\n""민주당 최고위원 보면 좀 비대칭 전력..."
3,정치,윤석열 대통령이 1일 서울 강서구 마곡동 ‘서울창업허브M+‘에서 열린 제5차 수출전...
4,정치,가상자산(암호화폐) 거래 논란으로 더불어민주당을 탈당한 김남국 무소속 의원이 31일...
...,...,...
584,경제,"코스피, 코스닥 상장사 5개 종목의 무더기 하한가 사태와 관련해 검찰이 온라인 주식..."
585,사회,15일 오후 5시24분쯤 대구 서구 중리동의 한 공장에서 원인을 알 수 없는 불이 ...
586,사회,기사와 직접 관련없는 자료사진. 동아일보 DB\n13일 오전 서울의 한 4년제 A ...
587,사회,캄보디아에서 숨진 한국인 인터넷방송 진행자(BJ) A 씨(여·33)의 사인 규명이 ...


In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

# Chrome 드라이버 옵션 설정
options = webdriver.ChromeOptions()

# 코랩에서는 headless 모드(화면에 브라우저가 표시되지 않는 모드)를 사용하면 안정적으로 실행됩니다.
options.add_argument('--headless') # 브라우저를 표시하지 않는 headless 모드로 실행합니다.
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def get_news_by_theme(theme_url, date):
    # 뉴스 테마와 본문을 저장할 리스트 초기화
    themes = []
    news_contents = []

    # 웹 드라이버 시작
    driver = webdriver.Chrome(options=options)

    # 대상 페이지로 이동
    driver.get(theme_url.format(date))
    time.sleep(3)

    # 10개의 뉴스 링크 가져오기
    news_links = []
    for i in range(1, 11):
        news_xpath = f'//*[@id="contents"]/div/div/div[2]/section/ul/li[{i}]/article/div/h4/a'
        news_element = driver.find_element(By.XPATH, news_xpath)
        news_link = news_element.get_attribute("href")
        news_links.append(news_link)

    # 10개의 뉴스 테마와 본문 내용 가져오기
    for news_link in news_links:
        driver.get(news_link)
        time.sleep(2)  # 페이지 로드를 기다리기 위해 충분한 시간을 줍니다.

        try:
            # 뉴스 테마 가져오기
            news_theme_element = driver.find_element(By.XPATH, '//*[@id="contents"]/header/div/section/nav/ol/li/a')
            news_theme = news_theme_element.text
            themes.append(news_theme)

            # 뉴스 본문 내용 가져오기
            news_content_element = driver.find_element(By.XPATH, "//*[@id=\"contents\"]/div[2]/div/div[1]/section[1]")
            news_content = news_content_element.text
            news_contents.append(news_content)

        except:
            print("테마나 본문 내용을 가져올 수 없습니다.")

    # 웹 드라이버 종료
    driver.quit()

    # 데이터프레임으로 변환하여 반환
    df = pd.DataFrame({"Theme": themes, "News Content": news_contents})
    return df

# 대상 페이지들의 URL
urls = {
    "Economy": "https://www.donga.com/news/Economy/List?ymd={}&m=",
}

# 날짜 설정
start_date = datetime(2023, 7, 1)
end_date = start_date + timedelta(days=4)  # 15일치

# 날짜별로 가져온 데이터프레임을 저장할 리스트 초기화
all_dfs = []

# 각 날짜에 대해 뉴스 가져오기
for single_date in pd.date_range(start=start_date, end=end_date):
    date_str = single_date.strftime("%Y%m%d")
    print("날짜:", date_str)

    # 각 테마별로 뉴스 가져오기
    dfs = []
    for theme, url in urls.items():
        print(f"{theme} 뉴스:")
        df = get_news_by_theme(url, date_str)
        dfs.append(df)
        print(df)
        print("="*50)

    # 각 날짜별 데이터프레임을 합쳐서 리스트에 저장
    all_dfs.append(pd.concat(dfs, ignore_index=True))

# 모든 날짜별 데이터프레임을 하나로 합치기
copy_df = pd.concat(all_dfs, ignore_index=True)

# 결과 출력
print("전체 뉴스:")
print(copy_df)


날짜: 20230701
Economy 뉴스:
  Theme                                       News Content
0    사회  '1, 6, 20, 27, 28, 41'…'15'\n2등 78명, 5473만7530...
1    경제  "물가안정" 농심·삼양식품·오뚜기·팔도 '라면 빅4' 13년만에 가격 내려\n풀무원...
2    경제  돈, 오카네, 머니. 세상 그 누가 돈에서 자유로울까요. 동전도 지폐도. 돈은 뒤집...
3    사회  여성가족부 6월 기준 양육비 제재 조치 통계\n남자 611명·여자 66명…18년간 ...
4    경제  “무엇보다도 관광산업은 일자리를 창출하는 최고의 산업이다. 자체적인 크루즈 상품을 ...
5    경제  현대건설 역대급 6.5조 석유화학단지 수주 잭팟\nHD현대 첫 해외 엔진 생산 공장...
6    국제  애플 일일 주가추이 - 야후 파이낸스 갈무리\n애플이 인류역사상 최초로 시총 3조 ...
7    경제  지난달 12일 부산항 신선대부두와 감만부두 야적장에 컨테이너가 가득 쌓여 있다. 2...
8    경제  12일 부산항 신선대부두 야적장에 컨테이너가 가득 쌓여 있다. 관세청이 이날 발표한...
9    경제  일본 등 단거리 여행수요 폭증에 LCC 항공사 긍정적 전망\n여름 휴가철 해외여행 ...
날짜: 20230702
Economy 뉴스:
  Theme                                       News Content
0    경제  제로 음료 사진. 게티이미지뱅크\n단맛을 내는 인공감미료 ‘아스파탐’이 발암 물질로...
1    정치  크게보기\n북한이 현정은 현대그룹 회장(68·사진)의 방북 추진에 대해 “남조선 어...
2    경제  크게보기\n델타항공 운항고객센터(OCC)의 기상 학자들. 변종국 기자 bjk@don...
3    경제  경기 부진이 지속되며 자영업자 10명 중 6명은 올 상반기(1~6월) 매출이 지난해...
4    문

In [11]:
# 두 데이터프레임을 수직으로 결합하여 새로운 데이터프레임 생성
new_all_news_df = pd.concat([all_news_df, copy_df], axis=0, ignore_index=True)

# 결과 출력
print(new_all_news_df)

    Theme                                       News Content
0      정치  장거리지대공미사일(L-SAM) 요격미사일이 발사되는 모습. ADD 제공\n북한이 군...
1      정치  불법 대선 자금 수수 혐의를 받고 있는 김용 전 민주연구원 부원장이 1일 오전 서울...
2      정치  "유력 이용호가 '최고위원 역할 없다'고"\n"민주당 최고위원 보면 좀 비대칭 전력...
3      정치  윤석열 대통령이 1일 서울 강서구 마곡동 ‘서울창업허브M+‘에서 열린 제5차 수출전...
4      정치  가상자산(암호화폐) 거래 논란으로 더불어민주당을 탈당한 김남국 무소속 의원이 31일...
..    ...                                                ...
631    경제  새로 출시된 무풍 시스템에어컨 인피니트 라인\n2017년 첫 선보인 후 6년만에 스...
632    경제  국회 기획재정위원회 전체회의서 답변\n추경호 경제부총리 겸 기획재정부 장관은 5일 ...
633    경제  CJ프레시웨이가 미국 유명 3대 버거 브랜드로 꼽히는 파이브가이즈의 국내 매장에 농...
634    경제  정부는 5일 윤석열 대통령 지시인 ‘킬러규제 혁신 TF‘를 꾸리고 분야별 킬러규제를...
635    경제  KCC글라스가 세 번째 ESG보고서를 발간했다.\n\nKCC글라스는 ‘2022‧20...

[636 rows x 2 columns]


In [13]:
new_all_news_df['Theme'].value_counts()

사회       173
정치       158
국제       142
경제       136
문화        10
스포츠        8
IT/의학      5
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [18]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

# Chrome 드라이버 옵션 설정
options = webdriver.ChromeOptions()

# 코랩에서는 headless 모드(화면에 브라우저가 표시되지 않는 모드)를 사용하면 안정적으로 실행됩니다.
options.add_argument('--headless') # 브라우저를 표시하지 않는 headless 모드로 실행합니다.
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def get_news_by_theme(theme_url, date):
    # 뉴스 테마와 본문을 저장할 리스트 초기화
    themes = []
    news_contents = []

    # 웹 드라이버 시작
    driver = webdriver.Chrome(options=options)

    # 대상 페이지로 이동
    driver.get(theme_url.format(date))
    time.sleep(3)

    # 10개의 뉴스 링크 가져오기
    news_links = []
    for i in range(1, 11):
        news_xpath = f'//*[@id="contents"]/div/div/div[2]/section/ul/li[{i}]/article/div/h4/a'
        news_element = driver.find_element(By.XPATH, news_xpath)
        news_link = news_element.get_attribute("href")
        news_links.append(news_link)

    # 10개의 뉴스 테마와 본문 내용 가져오기
    for news_link in news_links:
        driver.get(news_link)
        time.sleep(2)  # 페이지 로드를 기다리기 위해 충분한 시간을 줍니다.

        try:
            # 뉴스 테마 가져오기
            news_theme_element = driver.find_element(By.XPATH, '//*[@id="contents"]/header/div/section/nav/ol/li/a')
            news_theme = news_theme_element.text
            themes.append(news_theme)

            # 뉴스 본문 내용 가져오기
            news_content_element = driver.find_element(By.XPATH, "//*[@id=\"contents\"]/div[2]/div/div[1]/section[1]")
            news_content = news_content_element.text
            news_contents.append(news_content)

        except:
            print("테마나 본문 내용을 가져올 수 없습니다.")

    # 웹 드라이버 종료
    driver.quit()

    # 데이터프레임으로 변환하여 반환
    df = pd.DataFrame({"Theme": themes, "News Content": news_contents})
    return df

# 대상 페이지들의 URL
urls = {
    "Economy": "https://www.donga.com/news/Economy/List?ymd={}&m=",
}

# 날짜 설정
start_date = datetime(2023, 7, 1)
end_date = start_date + timedelta(days=4)  # 15일치

# 날짜별로 가져온 데이터프레임을 저장할 리스트 초기화
all_dfs = []

# 각 날짜에 대해 뉴스 가져오기
for single_date in pd.date_range(start=start_date, end=end_date):
    date_str = single_date.strftime("%Y%m%d")
    print("날짜:", date_str)

    # 각 테마별로 뉴스 가져오기
    dfs = []
    for theme, url in urls.items():
        print(f"{theme} 뉴스:")
        df = get_news_by_theme(url, date_str)
        dfs.append(df)
        print(df)
        print("="*50)

    # 각 날짜별 데이터프레임을 합쳐서 리스트에 저장
    all_dfs.append(pd.concat(dfs, ignore_index=True))

# 모든 날짜별 데이터프레임을 하나로 합치기
copy_df = pd.concat(all_dfs, ignore_index=True)

# 결과 출력
print("전체 뉴스:")
print(copy_df)

날짜: 20230701
Economy 뉴스:
  Theme                                       News Content
0    사회  '1, 6, 20, 27, 28, 41'…'15'\n2등 78명, 5473만7530...
1    경제  "물가안정" 농심·삼양식품·오뚜기·팔도 '라면 빅4' 13년만에 가격 내려\n풀무원...
2    경제  돈, 오카네, 머니. 세상 그 누가 돈에서 자유로울까요. 동전도 지폐도. 돈은 뒤집...
3    사회  여성가족부 6월 기준 양육비 제재 조치 통계\n남자 611명·여자 66명…18년간 ...
4    경제  “무엇보다도 관광산업은 일자리를 창출하는 최고의 산업이다. 자체적인 크루즈 상품을 ...
5    경제  현대건설 역대급 6.5조 석유화학단지 수주 잭팟\nHD현대 첫 해외 엔진 생산 공장...
6    국제  애플 일일 주가추이 - 야후 파이낸스 갈무리\n애플이 인류역사상 최초로 시총 3조 ...
7    경제  지난달 12일 부산항 신선대부두와 감만부두 야적장에 컨테이너가 가득 쌓여 있다. 2...
8    경제  12일 부산항 신선대부두 야적장에 컨테이너가 가득 쌓여 있다. 관세청이 이날 발표한...
9    경제  일본 등 단거리 여행수요 폭증에 LCC 항공사 긍정적 전망\n여름 휴가철 해외여행 ...
날짜: 20230702
Economy 뉴스:
  Theme                                       News Content
0    경제  제로 음료 사진. 게티이미지뱅크\n단맛을 내는 인공감미료 ‘아스파탐’이 발암 물질로...
1    정치  크게보기\n북한이 현정은 현대그룹 회장(68·사진)의 방북 추진에 대해 “남조선 어...
2    경제  크게보기\n델타항공 운항고객센터(OCC)의 기상 학자들. 변종국 기자 bjk@don...
3    경제  경기 부진이 지속되며 자영업자 10명 중 6명은 올 상반기(1~6월) 매출이 지난해...
4    문

In [19]:
# 두 데이터프레임을 수직으로 결합하여 새로운 데이터프레임 생성
new_all_news_df = pd.concat([new_all_news_df, copy_df], axis=0, ignore_index=True)

# 결과 출력
print(new_all_news_df)

    Theme                                       News Content
0      정치  장거리지대공미사일(L-SAM) 요격미사일이 발사되는 모습. ADD 제공\n북한이 군...
1      정치  불법 대선 자금 수수 혐의를 받고 있는 김용 전 민주연구원 부원장이 1일 오전 서울...
2      정치  "유력 이용호가 '최고위원 역할 없다'고"\n"민주당 최고위원 보면 좀 비대칭 전력...
3      정치  윤석열 대통령이 1일 서울 강서구 마곡동 ‘서울창업허브M+‘에서 열린 제5차 수출전...
4      정치  가상자산(암호화폐) 거래 논란으로 더불어민주당을 탈당한 김남국 무소속 의원이 31일...
..    ...                                                ...
677    경제  새로 출시된 무풍 시스템에어컨 인피니트 라인\n2017년 첫 선보인 후 6년만에 스...
678    경제  국회 기획재정위원회 전체회의서 답변\n추경호 경제부총리 겸 기획재정부 장관은 5일 ...
679    경제  CJ프레시웨이가 미국 유명 3대 버거 브랜드로 꼽히는 파이브가이즈의 국내 매장에 농...
680    경제  정부는 5일 윤석열 대통령 지시인 ‘킬러규제 혁신 TF‘를 꾸리고 분야별 킬러규제를...
681    경제  KCC글라스가 세 번째 ESG보고서를 발간했다.\n\nKCC글라스는 ‘2022‧20...

[682 rows x 2 columns]


In [20]:
new_all_news_df['Theme'].value_counts()

사회       182
경제       164
정치       159
국제       142
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [21]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

# Chrome 드라이버 옵션 설정
options = webdriver.ChromeOptions()

# 코랩에서는 headless 모드(화면에 브라우저가 표시되지 않는 모드)를 사용하면 안정적으로 실행됩니다.
options.add_argument('--headless') # 브라우저를 표시하지 않는 headless 모드로 실행합니다.
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def get_news_by_theme(theme_url, date):
    # 뉴스 테마와 본문을 저장할 리스트 초기화
    themes = []
    news_contents = []

    # 웹 드라이버 시작
    driver = webdriver.Chrome(options=options)

    # 대상 페이지로 이동
    driver.get(theme_url.format(date))
    time.sleep(3)

    # 10개의 뉴스 링크 가져오기
    news_links = []
    for i in range(1, 11):
        news_xpath = f'//*[@id="contents"]/div/div/div[2]/section/ul/li[{i}]/article/div/h4/a'
        news_element = driver.find_element(By.XPATH, news_xpath)
        news_link = news_element.get_attribute("href")
        news_links.append(news_link)

    # 10개의 뉴스 테마와 본문 내용 가져오기
    for news_link in news_links:
        driver.get(news_link)
        time.sleep(2)  # 페이지 로드를 기다리기 위해 충분한 시간을 줍니다.

        try:
            # 뉴스 테마 가져오기
            news_theme_element = driver.find_element(By.XPATH, '//*[@id="contents"]/header/div/section/nav/ol/li/a')
            news_theme = news_theme_element.text
            themes.append(news_theme)

            # 뉴스 본문 내용 가져오기
            news_content_element = driver.find_element(By.XPATH, "//*[@id=\"contents\"]/div[2]/div/div[1]/section[1]")
            news_content = news_content_element.text
            news_contents.append(news_content)

        except:
            print("테마나 본문 내용을 가져올 수 없습니다.")

    # 웹 드라이버 종료
    driver.quit()

    # 데이터프레임으로 변환하여 반환
    df = pd.DataFrame({"Theme": themes, "News Content": news_contents})
    return df

# 대상 페이지들의 URL
urls = {
    "International": "https://www.donga.com/news/Inter/List?ymd={}&m=",
}

# 날짜 설정
start_date = datetime(2023, 6, 18)
end_date = start_date + timedelta(days=2)  # 15일치

# 날짜별로 가져온 데이터프레임을 저장할 리스트 초기화
all_dfs = []

# 각 날짜에 대해 뉴스 가져오기
for single_date in pd.date_range(start=start_date, end=end_date):
    date_str = single_date.strftime("%Y%m%d")
    print("날짜:", date_str)

    # 각 테마별로 뉴스 가져오기
    dfs = []
    for theme, url in urls.items():
        print(f"{theme} 뉴스:")
        df = get_news_by_theme(url, date_str)
        dfs.append(df)
        print(df)
        print("="*50)

    # 각 날짜별 데이터프레임을 합쳐서 리스트에 저장
    all_dfs.append(pd.concat(dfs, ignore_index=True))

# 모든 날짜별 데이터프레임을 하나로 합치기
copy_df = pd.concat(all_dfs, ignore_index=True)

# 결과 출력
print("전체 뉴스:")
print(copy_df)

날짜: 20230618
International 뉴스:
  Theme                                       News Content
0    국제  최고온도 43도 및 44.7도\n인도 최대 주 2곳에서 지난 며칠 동안 폭염으로 최...
1    국제  뉴시스\n18일 친강(秦剛) 중국 국무위원 겸 외교부장은 중국을 방문한 토니 블링컨...
2    정치  크게보기\n한덕수 국무총리가 18일 서울 종로구 총리공관에서 열린 고위당정협의회에서...
3    국제  IS와 연계된 무장단체의 공격을 받은 우간다 학교. 뉴시스\n극단주의 테러단체인 이...
4    국제  사진 출처 위키미디어\n영국이 비만 예방을 위해 10월 시행하려던 ‘정크푸드 1+1...
5    국제  젤렌스키, “러시아군 철수 뒤 협상”\n푸틴, 협상 초안 공개하며 우크라 탓\n남아...
6    국제  후쿠시마 지역 어민 반발은 여전\n크게보기\n18일 오후 일본 도쿄 시나가와구 대형...
7    정치  윤석열 대통령이 지난해 11월 17일 서울 용산구 한남동 관저에서 회담과 오찬 일정...
8    국제  축구 국가대표 선수 손준호. 2022.11.22 뉴스1\n중국 공안에 ‘비(非)국가...
9    국제  “모든 해답은 역사 속에 있다.” 초 단위로 넘치는 뉴스의 홍수 속에서 하나의 흐름...
날짜: 20230619
International 뉴스:
  Theme                                       News Content
0    국제  테라폼랩스 권도형 대표. 뉴시스\n올 3월 동유럽 발칸반도의 몬테네그로에서 위조 여...
1    국제  토니 블링컨 미국 국무장관(왼쪽)과 시진핑 중국 국가주석이 19일(현지 시간) 중국...
2    사회  테라폼랩스 권도형 대표. 뉴시스\n\n위조 여권 사용 혐의로 몬테네그로에서 붙잡혀 ...
3    정치  19일 오전 국회에서 열린 제407회 국회(임시회)제4차 본회의에서 이재명 더불

In [22]:
# 두 데이터프레임을 수직으로 결합하여 새로운 데이터프레임 생성
new_all_news_df = pd.concat([new_all_news_df, copy_df], axis=0, ignore_index=True)

# 결과 출력
print(new_all_news_df)

    Theme                                       News Content
0      정치  장거리지대공미사일(L-SAM) 요격미사일이 발사되는 모습. ADD 제공\n북한이 군...
1      정치  불법 대선 자금 수수 혐의를 받고 있는 김용 전 민주연구원 부원장이 1일 오전 서울...
2      정치  "유력 이용호가 '최고위원 역할 없다'고"\n"민주당 최고위원 보면 좀 비대칭 전력...
3      정치  윤석열 대통령이 1일 서울 강서구 마곡동 ‘서울창업허브M+‘에서 열린 제5차 수출전...
4      정치  가상자산(암호화폐) 거래 논란으로 더불어민주당을 탈당한 김남국 무소속 의원이 31일...
..    ...                                                ...
707    정치  조승환 해양수산부 장관이 20일 오후 서울 여의도 국회에서 열린 농림축산식품해양수산...
708    국제  오염수 방류 이후 ‘유언비어 피해 발생할 것’ 87.8%\n주민들 “피해 대책·안전...
709    사회  송은석 기자 silverstone@donga.com\n“2022년산 소금 20kg,...
710    국제  외무장관 "제재 접근법 강화…재건 요구 엄청날 것"\n재무장관 "러시아, 악의적 파...
711    국제  금리 인상 효과 안 먹혀…각국 어려운 선택해야\n"금리 효과가 나타날 때까지 충분히...

[712 rows x 2 columns]


In [23]:
new_all_news_df['Theme'].value_counts()

사회       185
정치       167
경제       165
국제       160
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [30]:
new_all_news_df=new_all_news_df.drop(new_all_news_df[new_all_news_df['Theme'] == '사회'].index[:35])
new_all_news_df['Theme'].value_counts()

정치       167
경제       165
국제       160
사회       150
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [31]:
new_all_news_df=new_all_news_df.drop(new_all_news_df[new_all_news_df['Theme'] == '정치'].index[:17])
new_all_news_df['Theme'].value_counts()

경제       165
국제       160
정치       150
사회       150
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [32]:
new_all_news_df=new_all_news_df.drop(new_all_news_df[new_all_news_df['Theme'] == '경제'].index[:15])
new_all_news_df['Theme'].value_counts()

국제       160
정치       150
경제       150
사회       150
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [33]:
new_all_news_df=new_all_news_df.drop(new_all_news_df[new_all_news_df['Theme'] == '국제'].index[:10])
new_all_news_df['Theme'].value_counts()

정치       150
국제       150
경제       150
사회       150
IT/의학     11
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [34]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != 'IT/의학']
new_all_news_df['Theme'].value_counts()

정치       150
국제       150
경제       150
사회       150
스포츠       10
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [35]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != '스포츠']
new_all_news_df['Theme'].value_counts()

정치       150
국제       150
경제       150
사회       150
문화        10
사람속으로      2
오피니언       1
연예         1
Name: Theme, dtype: int64

In [36]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != '사람속으로']
new_all_news_df['Theme'].value_counts()

정치      150
국제      150
경제      150
사회      150
문화       10
오피니언      1
연예        1
Name: Theme, dtype: int64

In [37]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != '오피니언']
new_all_news_df['Theme'].value_counts()

정치    150
국제    150
경제    150
사회    150
문화     10
연예      1
Name: Theme, dtype: int64

In [38]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != '연예']
new_all_news_df['Theme'].value_counts()

정치    150
국제    150
경제    150
사회    150
문화     10
Name: Theme, dtype: int64

In [39]:
new_all_news_df=new_all_news_df[new_all_news_df['Theme'] != '문화']
new_all_news_df['Theme'].value_counts()

정치    150
국제    150
경제    150
사회    150
Name: Theme, dtype: int64

In [40]:
new_all_news_df.to_csv('new_all_news.csv', index=False)

In [59]:
df.head()

,Theme,News Content
0,정치,MB 정권 시절 언론 자유 현격히 낮춘 당사자\n정의당은 이동관 전 청와대 홍보수...
1,정치,크게보기\n북한 조선중앙통신이 제공한 사진에 지난달 북한 평안북도 동창리의 위성 ...
2,정치,우즈베키스탄을 공식 방문 중인 박진 외교부 장관은 현지시간 미르지요예프 대통령을 예...
3,국제,크게보기\nAP뉴시스\n\n\n 미국 비농업부문 고용이 만건으로 시장 전망치 만 건...
4,국제,괌을 강타하고 북상한 마와르 영향으로 본에서 폭우가 내리면서 강이 범람하고 산사태 ...


In [73]:
import pandas as pd
import numpy as np
import re

def preprocess_text(text):
    # 이메일 주소 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # http(s) 또는 www로 시작하는 링크 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # 유니코드 색상 코드 제거
    text = re.sub(r'#[0-9a-fA-F]{6}', '', text)
    # 숫자, 문자를 제외한 모든 것 제거 (이모티콘, 한자, 특수문자 등)
    text = re.sub(r'[^\w\s]', '', text)
    # 괄호 제거
    text = re.sub(r'[\(\)\·]', '', text)
    # 연속된 마침표 제거
    text = re.sub(r'\.{2,}', '', text)
    # "기자" 단어 제거
    text = re.sub(r'기자', '', text)
    # 숫자를 찾아서 제거
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[연년월일]', '', text)
    # '\n'을 빈 문자열로 대체하여 제거
    text = text.replace('\n', '')
    # '\r'을 빈 문자열로 대체하여 제거 (윈도우 환경에서 발생할 수 있음)
    text = text.replace('\r', '')
    # '\r\n'을 빈 문자열로 대체하여 제거 (윈도우 환경에서 발생할 수 있음)
    text = text.replace('\r\n', '')
    # '\t'을 빈 문자열로 대체하여 제거 (탭 문자)
    text = text.replace('\t', '')
    text = text.replace('서울뉴시스', '')
    text = text.replace('크게보기', '')
    return text


def remove_japanese_characters(text):
    # 일본어 문자 범위를 사용하여 일본어를 제거
    # 유니코드 범위:
    #   - 히라가나: 3040-309F
    #   - 카타카나: 30A0-30FF
    #   - 한자: 4E00-9FAF (일부 한자는 다른 범위에 속할 수 있음)
    # 이외에도 여러 범위가 더 있지만, 기본적인 예제로는 이 정도가 적당함
    cleaned_text = re.sub(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]+', '', text)
    return cleaned_text

# 예를 들어, df가 다음과 같이 정의되어 있다고 가정합니다.
# df = pd.DataFrame({'text': ['이것은 예시 문장입니다. https://example.com', np.nan, 123, '또 다른 예시 문장입니다.']})

# 데이터프레임의 각 행에 대해 전처리를 수행합니다.
for index in range(len(df['News Content'])):
    data = df['News Content'][index]
    # 데이터 타입이 문자열이 아닌 경우를 처리
    if not isinstance(data, str):
        data = str(data)  # 숫자나 NaN 등을 문자열로 변환
    preprocessed_text = preprocess_text(data)
    preprocessed_text = remove_japanese_characters(preprocessed_text)
    df.loc[index, 'News Content'] = preprocessed_text


In [81]:
df.to_csv('freamwork97.csv', index=False)

In [82]:
df

,Theme,News Content
0,정치,MB 정권 시절 언론 자유 현격히 낮춘 당사자정의당은 이동관 전 청와대 홍보수석이...
1,정치,북한 조선중앙통신이 제공한 사진에 지난달 북한 평안북도 동창리의 위성 발사장에서 ...
2,정치,우즈베키스탄을 공식 방문 중인 박진 외교부 장관은 현지시간 미르지요예프 대통령을 예...
3,국제,AP뉴시스 미국 비농업부문 고용이 만건으로 시장 전망치 만 건을 큰 폭으로 상회해 ...
4,국제,괌을 강타하고 북상한 마와르 영향으로 본에서 폭우가 내리면서 강이 범람하고 산사태 ...
...,...,...
595,정치,조승환 해양수산부 장관이 오후 서울 여의도 국회에서 열린 농림축산식품해양수산위원회...
596,국제,오염수 방류 이후 유언비어 피해 발생할 것 주민들 피해 대책안전성 확보 아직 미흡 ...
597,사회,송은석 산 소금 kg 만 원에 판매합니다 한 온라인 중고거래 플랫폼에는 지난해 생...
598,국제,외무장관 제재 접근법 강화재건 요구 엄청날 것재무장관 러시아 악의적 파괴에 재정적 ...
